# 1 - Preliminaries

In [ ]:
import copy
import itertools
from collections import defaultdict
from operator import itemgetter

#### Our dataset format
An event is a string. At each time instant, there may be several events, which is called an eventset.

An eventset is a list of events.

A sequence is a list of eventsets.

A dataset is a list of sequences.

Thus, a dataset is a list of lists of lists of strings.

In [ ]:
dataset =  [
    [["a"], ["a", "b", "c"], ["a", "c"], ["c"]],
    [["a"], ["c"], ["b", "c"]],
    [["a", "b"], ["d"], ["c"], ["b"], ["c"]],
    [["a"], ["c"], ["b"], ["c"]]
]

# 2- Foundations
### Subsequences

In [ ]:
"""
This is a simple recursive method that checks if subsequence is a subSequence of mainSequence
"""
def isSubsequence(mainSequence, subSequence):
    subSequenceClone = list(subSequence) # clone the sequence, because we will alter it
    return isSubsequenceRecursive(mainSequence, subSequenceClone) #start recursion

"""
Function for the recursive call of isSubsequence, not intended for external calls
"""
def isSubsequenceRecursive(mainSequence, subSequenceClone, start=0):
    # Check if empty: End of recursion, all eventsets have been found
    if (not subSequenceClone):
        return True
    # retrieves element of the subsequence and removes is from subsequence 
    firstElem = set(subSequenceClone.pop(0))
    # Search for the first eventset...
    for i in range(start, len(mainSequence)):
        if (set(mainSequence[i]).issuperset(firstElem)):
            # and recurse
            return isSubsequenceRecursive(mainSequence, subSequenceClone, i + 1)
    return False

In [ ]:
aSequence = [["a"], ["b", "c"], ["d"], ["a", "e"]]

In [ ]:
isSubsequence(aSequence, [["a"], ["d"], ["e"]])

In [ ]:
isSubsequence(aSequence, [["a"], ["b", "c"], ["e"]])

In [ ]:
isSubsequence(aSequence, [["a"], ["b", "d"]])

### Length of an eventset

In [ ]:
"""
Computes the length of the pattern (subsequence) (sum of the length of the contained eventsets)
"""
def sequenceLength(sequence):
    return sum(len(i) for i in sequence)

In [ ]:
print(sequenceLength ([["a"], ["b", "c"], ["a"], ["b","c","d"]]))

### Support of a pattern

In [ ]:
"""
Computes the support of a pattern (subsequence) in a dataset
"""
def countSupport (dataset, candidateSequence):
    return sum(1 for seq in dataset if isSubsequence(seq, candidateSequence)) 

In [ ]:
dataset

In [ ]:
countSupport(dataset, [["b"]])

In [ ]:
countSupport(dataset, [["a"], ["b", "c"]])

# 3- PrefixSpan algorithm

### Project a sequence

In [ ]:
"""
Projects a sequence according to a given prefix, as done in PrefixSpan

Args:
    sequence: the sequence the projection is built from
    prefix: the prefix that is searched for in the sequence
    newEvent: if set to True, the first eventset is ignored
Returns:
    If the sequence does not contain the prefix, then None.
    Otherwise, a new sequence starting from the position of the prefix, including the eventset that includes the prefix
"""
def projectSequence(sequence, prefix, newEvent):
    result = None
    for i, eventset in enumerate(sequence):
        if result is None:
            if (not newEvent) or i > 0:
                if (all(x in eventset for x in prefix)):
                    result = [list(eventset)]
        else:
            result.append(copy.copy(eventset))
    return result

In [ ]:
seq = [["a"], ["b", "c"], ["a", "c"], ["c"]]
projectSequence(seq, ["b"], False)

In [ ]:
projectSequence(seq, ["a", "c"], False)

In [ ]:
projectSequence(seq, ["a"], False)

In [ ]:
projectSequence(seq, ["a"], True)

### Project a dataset

In [ ]:
"""
Projects a dataset according to a given prefix, as done in PrefixSpan

Args:
    dataset: the dataset the projection is built from
    prefix: the prefix that is searched for in the sequence
    newEvent: if set to True, the first eventset is ignored
Returns:
    A (potentially empty) list of sequences
"""
def projectDatabase(dataset, prefix, newEvent):
    projectedDB = []
    for sequence in dataset:
        seqProjected = projectSequence(sequence, prefix, newEvent)
        if not seqProjected is None:
            projectedDB.append(seqProjected)
    return projectedDB

In [ ]:
datasetProject = [
            [["a"], ["a", "b", "c"], ["a", "c"], ["d"], ["c", "f"]],
            [["a", "d"], ["c"], ["b", "c"], ["a", "e"]],
            [["e", "f"], ["a", "b"], ["d", "f"], ["d"], ["b"]],
            [["e"], ["g"], ["a", "f"], ["c"], ["b"], ["c"]]
        ]

In [ ]:
projectDatabase(datasetProject, ["c"], False)

### The main algorithm

#### Some more utility functions:

In [ ]:
"""
Generates a list of all items that are contained in a dataset
"""
def generateItems(dataset):
    return sorted(set ([item for sublist1 in dataset for sublist2 in sublist1 for item in sublist2]))

"""
Computes a defaultdict that maps each item in the dataset to its support
"""
def generateItemSupports(dataset, ignoreFirstEvent=False, prefix=[]):
    result = defaultdict(int)
    for sequence in dataset:
        if ignoreFirstEvent:
            sequence = sequence[1:]
        cooccurringItems = set()
        for eventset in sequence:
            if all(x in eventset for x in prefix):
                for item in eventset:
                    if not item in prefix:
                        cooccurringItems.add(item)
        for item in cooccurringItems:
            result [item] += 1
    return sorted(result.items())

#### Finally, the algorithm:

In [ ]:
"""
The PrefixSpan algorithm. Computes the frequent patterns in a sequence dataset for a given minFrequency

Args:
    dataset: A list of sequences, for which the frequent (sub-)sequences are computed
    minFrequency: The minimum support that makes a pattern (subsequence) frequent
Returns:
    A list of tuples (s, c), where s is a frequent pattern, and c is the count for that pattern
"""
def prefixSpan(dataset, minFrequency):
    result = []
    itemCounts = generateItemSupports(dataset)
    for item, count in itemCounts:
        if count >= minFrequency:
            newPrefix = [item]
            result.append((newPrefix, count))
            result.extend(prefixSpanInternal(projectDatabase(dataset, [item], False), minFrequency, newPrefix))
    return result

def prefixSpanInternal(dataset, minFrequency, prevPrefixes=[]):
    result = []
    # Add a new event to the prefix
    itemCountSubsequentEvents = generateItemSupports(dataset, True)
    for item, count in itemCountSubsequentEvents:
        if count >= minFrequency:
            newPrefix = copy.deepcopy(prevPrefixes)
            newPrefix.append(item)
            result.append((newPrefix, count))
            result.extend(prefixSpanInternal(projectDatabase(dataset, [item], True), minFrequency, newPrefix))
    return result

In [ ]:
prefixSpan(dataset, 2)

# 4- Practical work: analyzing YahooFinance data
Now we try to find some episodes in a real world data sequence.

First, load the dataset.

In [ ]:
import csv

def loadYahooDataset():
    currentLineNumber=0
    idSeq='1'
    newEventSet=[]
    yahooDataset=[]

    for line in csv.reader((row for row in open("YahooFinance.data")), delimiter='\t'):
        currentLineNumber+=1
        if currentLineNumber==1:  # header line
            continue
        if line[1] != idSeq:
            yahooDataset.append(newEventSet)
            idSeq=line[1]
            newEventSet=[]
        if not line[2].endswith('0'):
            newEventSet.append(line[2])
    yahooDataset.append(newEventSet)
    return yahooDataset

In [ ]:
yahooDataset=loadYahooDataset()
yahooDataset